In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from PIL import Image

In [57]:
DATA_DIR = "./data/coco"

IMG_HEIGHT = 244
IMG_WIDTH = 244

SEED = 1

In [58]:
# Setup GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


1 Physical GPUs, 1 Logical GPUs


# Loading Data

In [60]:
# Load Image data
image_filenames = {}
image_dir = DATA_DIR + "/val2017/"

for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):
        image_filenames[int(filename[:-4])] = image_dir + filename

In [61]:
# Load bbox/annotations
with open(DATA_DIR + "/annotations/instances_val2017.json") as f:
    _ = json.load(f)
    categories = {entry["id"]:{
            "supercategory": entry["supercategory"],
            "name": entry["name"]
        } for entry in _["categories"]}
    instances = {entry["id"]:{
            "bbox": tuple(entry["bbox"]),
            "category_id": entry["category_id"],
            "image_id": entry["image_id"]
        } for entry in _["annotations"]}

# Env setup

In [62]:
from multiagent.infrastructure.env import ObjectLocalizationEnv, BetterRewardEnv, MovingEdgeEnv, HierarchicalZoomEnv, StretchyZoomEnv

In [63]:
model = tf.keras.models.Sequential(tf.keras.applications.VGG16(weights="imagenet", include_top = True).layers[:-2])

In [64]:
og_env = ObjectLocalizationEnv(model, (224, 224), feature_dim = 4096)
moving_edge_env = MovingEdgeEnv(model, (244, 244), feature_dim = 4096, transformation_factor = 0.4)
zoomzoom_env = HierarchicalZoomEnv(model, (244,244), feature_dim = 4096)
stretchyzoom_env = StretchyZoomEnv(model, (244,244), feature_dim = 4096)

# Training

In [65]:
from multiagent.infrastructure.trainer import RL_Trainer
from multiagent.agents.dqn_agent import DQN_Agent
from multiagent.util.dqn_utils import EpochSchedule

In [66]:
agent_params = { #TODO!!!!
    "replay_buffer_size": 500,
    "batch_size": 32,
    "gamma": 0.1,
    "epsilon": EpochSchedule(15, final_p = .1, initial_p = 1),
    "dropout": 0.1
}

In [67]:
params = {
    "learning_freq": 1,
    "seed": SEED,
    "agent_class": DQN_Agent,
    "optimizer": tf.keras.optimizers.Adam(learning_rate = 0.001),
    "loss": tf.keras.losses.Huber(),
    "agent_params": agent_params,
    "env": None, #PlaceHolder
    "max_path_length": 200,
    "model_name": "austensux", #PlaceHolder
    "save_freq": 0 #Manually save after every epoch
}

## Curating category-specific data

In [81]:
CATEGORY_ID = 1
NUM_EPOCHS = 15

In [76]:
from multiagent.util.data import find_all_instances, find_image_ids_with_category, process_image_filename, stream_images

category_image_ids = find_image_ids_with_category(instances, CATEGORY_ID)
print("{} images that include at least one instance of '{}'".format(len(people_image_ids), categories[CATEGORY_ID]["name"]))

2693 images that include at least one instance of 'person'


In [77]:
category_images = stream_images(category_image_ids, image_filenames)

## Training Loop

In [78]:
from copy import deepcopy

In [101]:
current_agent_params = deepcopy(agent_params)

params['env'] = og_env
params['model_name'] = "caceido"
params['agent_params'] = current_agent_params

In [74]:
trainer = RL_Trainer(params=current_params)

In [119]:
for epoch_iter in range(NUM_EPOCHS):
    
#     if epoch_iter:
#         trainer.agent.q_func = tf.keras.models.load_model("models/{}/epoch{}".format(params['model_name'], epoch_iter - 1))
    
    avg_returns = []
    avg_losses = []
    current_image_iter = 0
    for image_id, image_tensor in category_images:
        current_image_iter += 1
        print("\n\n####### Image {} of {} in Epoch {}".format(current_image_iter, len(category_image_ids), epoch_iter))
        
        keys, target_bboxs = find_all_instances(instances, image_id, CATEGORY_ID)
        image_tensor = tf.expand_dims(image_tensor, 0)
        
        trainer.env.training_reset(target_bboxs, image_tensor)
        returns, losses = trainer.run_training_loop(1)
        avg_returns.append(np.average(returns))
        avg_losses.append(np.average(losses))
        
        print("{} unfound instances out of {}".format(len(trainer.env.target_bboxs), len(trainer.env.orig_target_bboxs)))
        
        
    trainer.agent.epsilon.increment_epoch()
    
    # Saving progress and metrics
    trainer.agent.q_func.save("models/{}/epoch{}/".format(params['model_name'], epoch_iter))
    np.save("models/{}/returns/epoch{}.npy".format(params['model_name'], epoch_iter), avg_returns)
    np.save("models/{}/losses/epoch{}.npy".format(params['model_name'], epoch_iter), avg_losses)
    
    
    



####### Image 1 of 2693 in Epoch 0
********** Iteration 0 ************
PATH:   	 [1, 6, 1, 1, 7, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 4, 6, 1, 6, 1, 1, 1, 1, 3, 6, 1, 5, 4, 5, 4, 0, 4, 1, 4, 7, 0, 1, 2, 1, 0, 1, 0, 7, 4, 6, 1, 1, 1, 1, 1, 4, 0, 1, 1, 0, 7, 1, 6, 4, 1, 1, 1, 1, 0, 0, 1, 0, 5, 0, 7, 1, 4, 0, 7, 4, 1, 0, 1, 1, 6, 4, 1, 1, 1, 1, 1, 4, 4, 1, 6, 0, 0, 1, 6, 4, 1, 7, 3, 1, 4, 1, 6, 4, 7, 0, 1, 1, 1, 1, 0, 1, 4, 1, 1, 1, 6, 1, 0, 7, 5, 7, 4, 1, 0, 0, 4, 1, 1, 4, 1, 1, 4, 6, 6, 4, 7, 0, 7, 4, 6, 1, 3, 2, 1, 4, 1, 4, 1, 7, 1, 6, 1, 1, 1, 4, 1, 1, 1, 1, 6, 1, 0, 1, 4, 1, 1, 5, 1, 4, 1, 1, 0, 1, 5, 4, 4, 7, 1, 0, 1, 1, 1, 0, 0, 1, 0, 7, 0, 7, 0, 1, 0, 7, 0, 1, 1, 1, 1, 0]
REWARDS:	 [-1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 

PATH:   	 [6, 1, 1, 1, 8, 1, 1, 1, 1, 8, 3, 1, 8, 1, 1, 0, 1, 0, 1, 1, 0, 8, 1, 8, 8, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 6, 1, 4, 8, 0, 6, 8, 8, 1, 1, 1, 1, 1, 1, 3, 6, 1, 1, 1, 1, 0, 1, 8, 3, 1, 8, 1, 1, 1, 1, 1, 1, 6, 4, 1, 1, 1, 8, 1, 1, 1, 8, 8, 1, 1, 1, 1, 3, 1, 8, 1, 6, 4, 8, 1, 8, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 3, 6, 8, 1, 1, 1, 0, 1, 1, 0, 3, 6, 1, 1, 4, 8, 1, 1, 6, 0, 1, 1, 0, 4, 8, 6, 1, 1, 4, 8, 1, 8, 3, 6, 8, 1, 6, 1, 1, 8, 8, 3, 6, 4, 8, 1, 3, 1, 1, 6, 1, 0, 1, 4, 8, 8, 6, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 4, 8, 1, 8, 1, 8, 6, 8, 1, 6, 8, 1, 8, 1, 0, 1, 1, 8, 0, 8, 8, 1, 1, 1, 3, 1, 6, 4, 1, 8, 1]
REWARDS:	 [1.0, -1.0, -1.0, -1.0, 3.0, -1.0, -1.0, -1.0, -1.0, 3.0, 1.0, -1.0, 3.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 3.0, -1.0, 3.0, 3.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 3.0, 1.0, 1.0, 3.0, 3.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 3.0, 1.0, -1.0, 3.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1

KeyboardInterrupt: 

In [87]:
category

ValueError: not enough values to unpack (expected 3, got 1)

In [89]:
image_tensor

<tf.Tensor: id=1684, shape=(1,), dtype=string, numpy=array([b'./data/coco/val2017/000000393226.jpg'], dtype=object)>

In [91]:
a, b = category_images.__next__()

In [94]:
d,e,f = b.shape